## Activity 31: Double Baby Block ##
Double baby block is a block cipher that works on 4 bit blocks. Any binary stream can be enciphered with this system. It is based on baby block which takes a 4-bit key. Double baby block enciphers first with an arbitrary 4-bit key and then with a second arbitrary key.

You have intercepted the following message from adversaries in a secret organization working closely with agents in a foreign government.

In [25]:
message = 'MNAGZQFCYD.U#TFYZCMWPJOBZIUV#Q?,SEI?CLERHTFH,OSR#BZTBC.RG@QHAJAERIMBWEGKWIZJBDIF,ZCMVJAFSHFASHGYAD!FT@OKZL,,YAPCQLFL.R\
EVMPEMGNCMZJCMQYXYGYTUIDF?BM@YIAZIA@YKYLQIZJYBS!,,WKGACFNASF,QSFHK,Z#YIE?WZINA.?G,S?NA#!YYJRYQEIZJAAWKBAFFF#GKGYH?#AJDUP,ICUAEG,P\
Y@IZK#IOKQIZIIRM'

You have reason to believe that the first letter of the ciphertext represents a "W" in the plaintext. In this activity, we will break the cipher using the Meet in the Middle Attack. Run the following cell to load every possible four digit key into a list called _potential keys_.

In [26]:
potential_keys = ['0000','0001','0010','0011','0100','0101','0110','0111','1000','1001','1010','1011','1100','1101',
                  '1110','1111']

W is 10110 in our binary alphabet and M is 01100, therefore we need to encipher W and _*decipher*_ M. Run the cell below so that we can organize our results into a table.

In [27]:
%pip install tabulate
from tabulate import tabulate

Note: you may need to restart the kernel to use updated packages.


Run the cells below to create a list of the resulting four bit strings that result from enciphering *1011*

In [28]:
def simple_baby_block(message,key):
 # both message and key are a four digit binary strings.
 # this program doesn't do any flipping, so the last two bits are always unchanged
    
  key = int((key),2)
  
  blocks = [message]

  sbox = [
          ['10', '10', '00', '11'], #0
          ['00', '11', '10', '00'],
          ['00', '10', '01', '00'],
          ['01', '00', '11', '11'],
          ['11', '00', '10', '01'],
          ['10', '11', '01', '10'],#5
          ['11', '01', '11', '10'],
          ['10', '01', '00', '01'],
          ['00', '01', '11', '10'],#8
          ['01', '00', '01', '11'],#9
          ['11', '11', '00', '00'],
          ['11', '10', '01', '01'],#11
          ['01', '11', '10', '10'],
          ['00', '01', '11', '00'],
          ['10', '00', '00', '01'],#14
          ['01', '10', '10', '11']]
  
  y = (sbox[key]) 
  r = blocks[0]
  sbox_output = r[2:4]
  sbox_output = sbox
  z = len(blocks)
  output = ''  
  for i in range(0,z):
    a = (blocks[i][2:4])
    b = int(a,2)
    b = y[b]
    c = str(int(b)^int(blocks[i][0:2]))
    while len(c) < 2:
        c = "0" + c
    c = c + blocks[i][2:4]    
    output += c
                     
  return(output)  

In [29]:
def simple_baby_block_thrice(message,key):
 # both message and key are a four digit binary strings.
 # this program swaps the first two and last two bits twice, after the first and second key are used
 
    key2 = key[1:4] + key[0] #rotates the key once
    key3 = key2[1:4] + key2[0] #rotates the key twice
    once = simple_baby_block(message,key)
    once = once[2:4] + once[0:2] #swap
    twice = simple_baby_block(once,key2)
    twice = twice[2:4] + twice[0:2] #swap
    thrice = simple_baby_block(twice,key3)

    return(thrice)   
  

In [30]:
encipher_list = []
for i in range(16):
 encipher_list.append(simple_baby_block_thrice("1011",potential_keys[i]))
print(encipher_list)

['1101', '0010', '1101', '1110', '0001', '1000', '0110', '0010', '0011', '1111', '1010', '1110', '1110', '1010', '1011', '1101']


The letter M is represented in our alphabet as _01100_, but for our purposes we only need decipher the first four bits. Run the cells below to create a list that results from deciphering "M" with baby block.

In [31]:
def simple_unbaby_block_thrice(message, key):
    
    key2 = key[1:4] + key[0]
    key3 = key[2:4] + key[0:2]
    a = simple_baby_block(message,key3)
    a = a[2:4] + a[0:2]
    b = simple_baby_block(a,key2)
    b = b[2:4] + b[0:2]
    c = simple_baby_block(b,key)
    
    return(c)
    

In [32]:
decipher_list = []
for i in range(16):
    decipher_list.append(simple_unbaby_block_thrice("0110",potential_keys[i]))
print(decipher_list)    

['1100', '0110', '1000', '1000', '0000', '1101', '1011', '1010', '1110', '1001', '1001', '0011', '0011', '0100', '0010', '0101']


Now, run the cell below to create a table to organize this information and look for keys that meet in the middle.

In [33]:
mydata = []
for i in range(0,16):
    mydata.append([potential_keys[i],encipher_list[i],'     ',decipher_list[i]])
print((mydata))
print(tabulate(mydata))



[['0000', '1101', '     ', '1100'], ['0001', '0010', '     ', '0110'], ['0010', '1101', '     ', '1000'], ['0011', '1110', '     ', '1000'], ['0100', '0001', '     ', '0000'], ['0101', '1000', '     ', '1101'], ['0110', '0110', '     ', '1011'], ['0111', '0010', '     ', '1010'], ['1000', '0011', '     ', '1110'], ['1001', '1111', '     ', '1001'], ['1010', '1010', '     ', '1001'], ['1011', '1110', '     ', '0011'], ['1100', '1110', '     ', '0011'], ['1101', '1010', '     ', '0100'], ['1110', '1011', '     ', '0010'], ['1111', '1101', '     ', '0101']]
----  ----    ----
0000  1101    1100
0001  0010    0110
0010  1101    1000
0011  1110    1000
0100  0001    0000
0101  1000    1101
0110  0110    1011
0111  0010    1010
1000  0011    1110
1001  1111    1001
1010  1010    1001
1011  1110    0011
1100  1110    0011
1101  1010    0100
1110  1011    0010
1111  1101    0101
----  ----    ----


Find matching pairs in the second and third columns. One possible eight digit key will consist of the four bits immediately to the right of the four bits in the second column paired with the four bits two to the left of the match in the third column.

For example, find 1101 at the top of the second column and sixth down on the third column. One possible key is "0000 0101".

Use the table to find every possible key and enter them into the cell below in a list called _possible_.

Hint: There are 15 (unless I missed one).

In [34]:
possible = ["00000101",...]

Run the cell below. Use the function to run all fifteen possible keys with our message and decipher our message.

In [35]:
def unbaby_block(message,key,LETTERS = "ABCDEFGHIJKLMNOPQRSTUVWXYZ,.?!@#"):
  output = ''
  outputfinal = ''
  message = message.upper()
  cl_message = ''  
  for char in message:
    if char in LETTERS:
      cl_message += char
  binary = {"A":"00000","B":"00001","C":"00010", "D":"00011","E":"00100","F":"00101","G":"00110",
            "H":"00111","I":"01000","J":"01001","K":"01010","L":"01011","M":"01100","N":"01101",
            "O":"01110","P":"01111","Q":"10000","R":"10001","S":"10010","T":"10011","U":"10100",
            "V":"10101","W":"10110","X":"10111","Y":"11000","Z":"11001",",":"11010",".":"11011",
            "?":"11100","!":"11101","@":"11110","#":"11111"}
  rev_binary = {v: k for k, v in binary.items()}
  binary_message = ""
  for char in cl_message:
    binary_message += binary[char]  
  blocks = []
  for i in range(0,len(binary_message),4):
    blocks.append(binary_message[i:i+4])
    
  for j in range(0,len(blocks)):
    a = blocks[j]
    a = simple_baby_block(a,key[6:8]+key[4:6])
    a = a[2:] + a[0:2]
    b = simple_baby_block(a,key[5:8]+key[4])
    b = b[2:] + b[0:2]
    c = simple_baby_block(b,key[4:8])
    d = simple_baby_block(c,key[2:4]+key[0:2])
    d = d[2:] + d[0:2]
    e = simple_baby_block(d,key[1:4]+key[0])
    e = e[2:] + e[0:2]
    f = simple_baby_block(e,key[0:4])
    output += f
  for k in range(0,len(output),5):
    outputfinal += rev_binary[output[k:k+5]]
  return(outputfinal.lower())  
    

In [ ]:
for i in range(15):
    ...      

# To receive credit for this assignment, post one or two words from the plaintext to the Activity 31 on EdStem.